In [1]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
x = [1,2,3,4]
y = [9,8,7,6]

m,b = np.polyfit(x, y, 1)
m

-0.9999999999999997

In [0]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/rubber"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#set the parameters
dataset_class = "validation"
source_file =PATH+ '/input_mix/validation/raw/DATA_25_04.csv'
TARGET_DIR = PATH +'/input_mix/training/cleaned/' #default target directory

In [0]:
original_data = pd.read_csv(source_file)
#original_data[100:200]
print(original_data)

     NATURAL_RUBBER_EXPORT  RSS3_PRICE  ...  Working Days  countryterritoryCode
0                194521581      265.50  ...            24                     0
1                182777386      282.00  ...            23                     0
2                195785072      281.00  ...            25                     0
3                201383861      289.75  ...            24                     0
4                210766103      306.75  ...            25                     0
..                     ...         ...  ...           ...                   ...
139              257767938      149.00  ...            25                     0
140              244023307      148.20  ...            24                     0
141              216770228      150.80  ...            27                     0
142              207468077      160.50  ...            25                     0
143              196128479      166.30  ...            24                     0

[144 rows x 22 columns]


In [0]:
def generate_synthetic_data(sample_dataset, window_mean, window_std, fixed_window=None,  sythesize_ratio = 2, forced_reverse = False):
    synthetic_data = pd.DataFrame(columns=sample_dataset.columns)
    synthetic_data.insert(len(sample_dataset.columns), "synthesis_seq", [], True) 

    
    for k in range(sythesize_ratio):
      if len(synthetic_data) >= len(sample_dataset) * sythesize_ratio:
        break;
      #this loop generates a set that resembles the entire dataset
      for country in sample_dataset['countryterritoryCode'].unique():
        country_synthetic = pd.DataFrame(columns=synthetic_data.columns)

        if fixed_window != None:
          input_sequence_len =  fixed_window
        else:
          input_sequence_len = int(np.random.normal(window_mean, window_std)) 

        #population data change
        country_data_i = sample_dataset[sample_dataset['countryterritoryCode'] == country]
        if len(country_data_i) < input_sequence_len :
          continue
        feature_length = configuration['feature_length']-2 #number of features to be randomized
        country_data_array = country_data_i.to_numpy()
        country_data_array = country_data_array.T[:feature_length]
        country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
        x = country_data_array[:feature_length].T
        
        reversed = np.random.normal(0,1)>0
        if reversed:
          x = x[::-1]
            
        sets =0
        x_list = []
        dict_x = dict()
        for i in range(input_sequence_len):
          array_len = ((len(x) -i) - ((len(x)-i)%input_sequence_len))+i
          if array_len <= 0:
            continue
          sets = int( array_len/ input_sequence_len)
          if sets <= 0:
            continue
          #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
          x_temp = x[i:array_len].T.reshape(sets,feature_length,input_sequence_len)
          uniq_keys = np.array([i+(input_sequence_len*k) for k in range(sets)])
          x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
          arrays_split = np.hsplit(x_temp,sets)
          dict_x.update(dict(zip(uniq_keys, arrays_split)))

        temp_x_list  = [dict_x[i].T for i in sorted(dict_x.keys())]        
        temp_x_list = np.array(temp_x_list).squeeze()
        feature_means = np.mean(temp_x_list, axis=1)
        feature_std = np.std(temp_x_list, axis=1)
        random_normal_feature_values = np.random.normal(feature_means, feature_std).T
        random_normal_feature_values = np.round(random_normal_feature_values,0)
        random_normal_feature_values[random_normal_feature_values < 0] = 0

        if reversed:
          random_normal_feature_values = random_normal_feature_values.T[::-1]
          random_normal_feature_values = random_normal_feature_values.T


        for column in country_data_i.columns:
          country_synthetic[column] = country_data_i[column]
        country_synthetic['synthesis_seq'] = k
        synthetic_data = synthetic_data.append(country_synthetic, ignore_index=True)
    return synthetic_data

In [0]:
configuration = {
    'input_sequence_len' : 6,
    'output_sequence_len' : 6,
    'training_batch_size' : 1,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1,
    'yhat_size' : 1,
    'feature_length' : 23,
    'output_size' : 1,
}

def verify_clean(cleaned_data):
  print(len(cleaned_data))
  print(cleaned_data.isna().any())
  print(cleaned_data.head())
sample_synthetic = generate_synthetic_data(original_data, window_mean = 0,  window_std= 0, fixed_window=3 ,sythesize_ratio = 14)
verify_clean(sample_synthetic)

2016
NATURAL_RUBBER_EXPORT          False
RSS3_PRICE                     False
RSS3_OPEN                      False
RSS3_HIGH                      False
RSS3_LOW                       False
RSS3_VOLUME                    False
TSR20_PRICE                    False
TSR20_OPEN                     False
TSR20_HIGH                     False
TSR20_LOW                      False
TSR20_VOLUME                   False
CARMI                          False
OIL_PRICE                      False
USD_ISD_EXCHANGE_RATE_PRICE    False
USD_ISD_EXCHANGE_RATE_OPEN     False
USD_ISD_EXCHANGE_RATE_HIGH     False
USD_ISD_EXCHANGE_RATE_LOW      False
Temperature - (Celsius)        False
Rainfall - (MM)                False
Ramadan Holidays               False
Working Days                   False
countryterritoryCode           False
synthesis_seq                  False
dtype: bool
  NATURAL_RUBBER_EXPORT  RSS3_PRICE  ...  countryterritoryCode  synthesis_seq
0             194521581      265.50  ...              

In [0]:
TARGET_DIR = PATH +'/input_mix/training/cleaned/'
run_id = 3042013 #ddmyyhh
run_id = 552022 #ddmyyhh
for i in range(10):
  #window_mean = i+6
  #window_std= 1
  synthetic_data  = generate_synthetic_data(original_data, window_mean = 0,  window_std= 0,fixed_window=2, sythesize_ratio = 100)
  synthetic_data.to_csv(TARGET_DIR + '/synthetic_run_'+str(i)+'_'+str(run_id)+'_dt_27_04'+'_window_'+str(2)+'.csv',  index=False )
  #'_wstd_'+str(window_std)+
  print('synth step : ', i, ' len : ', len(synthetic_data))

synth step :  0  len :  14400
synth step :  1  len :  14400
synth step :  2  len :  14400
synth step :  3  len :  14400
synth step :  4  len :  14400
synth step :  5  len :  14400
synth step :  6  len :  14400
synth step :  7  len :  14400
synth step :  8  len :  14400
synth step :  9  len :  14400


In [0]:
#testing for accuracy
def verify_synthetic_data(synthetic_data,original_data, print_results = False):
  countryterritoryCode ='CN'
  #test_synthetic_data = synthetic_data[synthetic_data['countryterritoryCode'] == countryterritoryCode]
  test_synthetic_data = synthetic_data
  control_set = original_data #[original_data['countryterritoryCode']==countryterritoryCode]
  case_mean = np.mean(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_mean = np.mean(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_mean = np.mean(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  case_std = np.std(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_std = np.std(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_std = np.std(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  if print_results:
    print(test_synthetic_data.head())
    print(control_set.head())
    print('original data length : ', len(control_set))
    print('synthetic data length : ',len(test_synthetic_data))
    print('min sequence length : ',min([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))
    print('max sequence length : ',max([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))

    print('Mean diff. cases : ', np.mean(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy()))
    print('Mean diff. deaths : ', np.mean(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy()))
    print('Mean diff. population : ', np.mean(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy()))
  
  return  [case_mean, death_mean, pop_mean, case_std , death_std , pop_std ]

In [0]:
"""" results = []
 for i in range(2,20):
   syn_data = generate_synthetic_data(original_data,i, sythesize_ratio = 1)
   print('window :', i, ' len :', len(syn_data))
   results.append(verify_synthetic_data(syn_data,original_data))

"""

'" results = []\n for i in range(2,20):\n   syn_data = generate_synthetic_data(original_data,i, sythesize_ratio = 1)\n   print(\'window :\', i, \' len :\', len(syn_data))\n   results.append(verify_synthetic_data(syn_data,original_data))\n\n'

In [0]:

for i in range(1):
  rand_number = np.random.normal(0,1)
  if rand_number >= 0:
    rand_number =1
  else:
    rand_number =0
  window_size = rand_number*(15 + np.round(np.random.normal(0,1))) + (1 - rand_number)*(4 + np.round(np.random.normal(0,1)))
  synthetic_data = generate_synthetic_data(original_data, run_id = str(i)+str(run_id), window = int(window_size), sythesize_ratio = 1)
  synthetic_data.to_csv(PATH+'/input/raw/synthetic/synthetic_'+str(i)+'_'+str(run_id)+'_dt27_04_'+str(int(window_size))+'.csv',  index=False )
  print('synth step : ', i, ' len : ', len(synthetic_data))

TypeError: ignored